In [1]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs']

In [2]:
def get64Samples(real, imag):

    # Number of signals
    num_signals = real.shape[0]  # 3 in this case
    
    # Initialize lists to store the focused samples
    imp_real_parts = []
    imp_imag_parts = []
    
    for i in range(num_signals):
        # Calculate the magnitude
        magnitude = np.abs(real[i] + 1j * imag[i])
        
        # find the peak index
        peak_index = np.argmax(magnitude)
        
        # Calculate the start and end indices for the focused part
        start_index = max(0, peak_index - 32)
        end_index = min(magnitude.shape[0], peak_index + 32)
        
        # Extract the part of the signal around the peak
        real_part_focus = real[i, start_index:end_index]
        imag_part_focus = imag[i, start_index:end_index]
        
        imp_real_parts.append(real_part_focus)
        imp_imag_parts.append(imag_part_focus)
        

    # Convert lists back to arrays for further processing if needed
    imp_real_parts = np.array(imp_real_parts)
    imp_imag_parts = np.array(imp_imag_parts)

    return imp_real_parts, imp_imag_parts

In [10]:
# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

# Extract data for ALICE and BOB channels
alice_CIRs = data_cir[:, alice_channel, :, :]  # Shape: (9797, 251, 2)
eve_CIRs = data_cir[:, eve_channel, :, :]  # Shape: (9797, 251, 2)

# ALICE features - real, imaginary, magnitude
alice_real_251 = alice_CIRs[:, :, 0]
alice_imag_251 = alice_CIRs[:, :, 1]
# Extract important 64 samples
alice_real, alice_imag = get64Samples(alice_real_251, alice_imag_251)
alice_features = np.hstack((alice_real, alice_imag))


# EVE features - real, imaginary, magnitude
eve_real_251 = eve_CIRs[:, :, 0]
eve_imag_251 = eve_CIRs[:, :, 1]
eve_real, eve_imag = get64Samples(eve_real_251, eve_imag_251)
eve_features = np.hstack((eve_real, eve_imag))

# print(alice_features.shape) # (9797, 753)
# print(eve_features.shape)

# # Atoms for our dictionary - Reshape Alice's data to flatten each sample.
# alice_atoms = alice_features.reshape(100, -1)  # Each sample becomes a long vector.
# eve_atoms = eve_features.reshape(100, -1)

# Create labels for Alice and Eve.
alice_labels = np.zeros(alice_features.shape[0])  # Label '0' for Alice. (9797,)
eve_labels = np.ones(alice_features.shape[0])     # Label '1' for Eve. (9797,)

print(alice_features.shape)
print(alice_labels.shape)

# # Combine data and labels.
atoms = np.vstack((alice_features, eve_features))  # (19594, 753)
true_labels = np.hstack((alice_labels, eve_labels))  # (19594,)


(9797, 128)
(9797,)


In [4]:
# Step 3: Split the Data into Training and Test Sets
trainData, testData, trainLabel, testLabel = train_test_split(atoms, true_labels, test_size=0.2, random_state=42, stratify=true_labels)

# Step 4: Form the Dictionary D
D = trainData.T

# Dictionary shape (128, 15675)
# trainData (15675, 128) -- > 80% of 19594
# testData (3919, 128)   -- > 20% of 19594
# trainLabel (15675,) -- > 80% of 19594
# testLabel (3919,)  -- > 20% of 19594

# print(D.shape)
# print(trainData.shape)
# print(testData.shape)
# print(trainLabel.shape)
# print(testLabel.shape)


In [5]:
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=10):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# D - (753, 160)
# tSample - (753,)

In [6]:
# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# tSample - (753,)
# coefficients - (160,)
# class_indices - (80,)
# reconstructed_signal - (753,)


In [7]:
# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    # tSample - (753,)
    # D - (753, 160)
    # trainLabel - (160,)
    # coefficients - (160,)
    
    # Find sparse coefficients for the new signal
    coefficients = find_sparse_coefficients(tSample, D)
    # print(coefficients)
    
    # Initialize residuals list
    residuals = []

    # Calculate residual for each class
    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class


In [8]:
# Step 8: Classifying Test Data and Evaluating the Model

# testData (40, 753)
predictions = []
for testSample in testData:
    predicted_class = classify_signal(testSample, D, trainLabel)
    predictions.append(predicted_class)


# Convert predictions to a numpy array for comparison
predictions = np.array(predictions)
# print(predictions)

# Step 9: Calculate Accuracy
accuracy = np.mean(predictions == testLabel)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

Classification Accuracy: 62.67%


In [9]:
# Calculate confusion matrix
print(f"\nTotal testing channel: {testData.shape}")
# print(true_labels.shape)
# print(predictions.shape)
tn, fp, fn, tp = confusion_matrix(testLabel, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Total testing channel: (3919, 128)
tp: 1590
tn: 866
fp: 1094
fn: 369
MDR: 0.5581632653061225
FAR: 0.1883614088820827
AR: 0.6267376629058974
